In [1]:
from flows import RealNVP, LightningINN
from autoencoders import VanillaVAE, LightningAutoEncoder
from utils import BoidImagesDataset, train_transform

import torch
from torch.utils.data import DataLoader
import lightning as L

/home/derin/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/derin/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:740: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [2]:
train_data_set = BoidImagesDataset("boid_data", transform=train_transform)
train_loader = DataLoader(train_data_set, 20, shuffle=True)

In [ ]:
from utils import ZeroChannelTransform, displayTensorImage
from PIL import Image
import matplotlib.pyplot as plt

for b in train_data_set:

    imgs: Image
    params, imgs = b

    displayTensorImage(imgs, plt)
    plt.show()

In [ ]:
vanilla_VAE = VanillaVAE(3, 500)
model = LightningAutoEncoder.load_from_checkpoint("vae_epoch_1094.ckpt", autoencoder=vanilla_VAE)

In [ ]:
real_NVP = RealNVP(8, 200, 5, 500)
lightning_inn = LightningINN(inn=real_NVP, encoder=vanilla_VAE.encode)

trainer = L.Trainer(max_time={"seconds": 20})
trainer.fit(model=lightning_inn, train_dataloaders= train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-03-11 14:56:32.748851: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 14:56:32.748929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 14:56:32.750796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 14:56:32.760733: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instruc

Epoch 0:  14%|█▍        | 139/1000 [00:19<02:03,  6.99it/s, v_num=15]

Time limit reached. Elapsed time is 0:00:20. Signaling Trainer to stop.


Epoch 0:  14%|█▍        | 140/1000 [00:20<02:04,  6.93it/s, v_num=15]
